In [114]:
# Seed a sqlite3 db with player stats (one table for different positions QB,TEAM,DEF_PLAYERS, KICKERS, FLEX)

import pandas as pd
import numpy as np

dynasty_df = pd.read_csv('870520789705854976.csv')
redraft_df1 = pd.read_csv('784961395996356608.csv')
redraft_df2 = pd.read_csv('849473673709629440.csv')

dynasty_starterIds = list(map(lambda x: x.split(','), dynasty_df['starterIds'].to_numpy()))
redraft1_starterIds = list(map(lambda x: x.split(','), redraft_df1['starterIds'].to_numpy()))
redraft2_starterIds = list(map(lambda x: x.split(','), redraft_df2['starterIds'].to_numpy()))

dynasty_starterIds = [j for i in dynasty_starterIds for j in i]
redraft1_starterIds = [j for i in redraft1_starterIds for j in i]
redraft2_starterIds = [j for i in redraft2_starterIds for j in i]

combinedIds = dynasty_starterIds + redraft_starterIds
uniquePlayerIds = np.unique(combinedIds)

395

In [2]:
# For Each Unique Player Identify their Position
# Retrieve Data and save somewhere...could create multiple data frames and save to csv before introducing the sqlite part of it...
import pandas as pd

players_map = dict(pd.read_json('../players.json')[['id', 'positions']].values)

In [115]:
# TODO need an easy way to import past years and new years...
# Could reseed by year? just in case previous stat changes...
# Need to update the python refresh script to only pull new matchups or merge with latest

# TODO need to run this again to get last nights scores
# TODO need an easy way to validate the stats are correct -> AUTOSTAT VERIFICATION...
import requests
import json
import time

def saveFile(filename, data):
    with open(filename, 'w') as datafile:
        json.dump(data, datafile)

BASE_STAT_URL = "https://api.sleeper.com/stats/nfl/player/"
YEAR = 2022
for playerId in uniquePlayerIds:
    stat_url = f'{BASE_STAT_URL}{playerId}?season_type=regular&season={YEAR}&grouping=week'
    response = requests.get(stat_url)
    saveFile(f'storage/{playerId}.csv', response.json())
    time.sleep(1)
    print(playerId, '/', len(uniquePlayerIds))


1052 / 395
1166 / 395
1213 / 395
1233 / 395
1234 / 395
1264 / 395
1266 / 395
1339 / 395
1343 / 395
1348 / 395
1352 / 395
1373 / 395
1426 / 395
1433 / 395
1466 / 395
1476 / 395
1479 / 395
1518 / 395
1535 / 395
167 / 395
1689 / 395
17 / 395
1837 / 395
1875 / 395
1905 / 395
1945 / 395
1992 / 395
1999 / 395
2020 / 395
2028 / 395
2036 / 395
2064 / 395
2133 / 395
2152 / 395
2161 / 395
2197 / 395
2216 / 395
2227 / 395
2251 / 395
2307 / 395
2309 / 395
2311 / 395
2320 / 395
2325 / 395
2330 / 395
2331 / 395
2350 / 395
2374 / 395
2389 / 395
2393 / 395
24 / 395
2427 / 395
2445 / 395
2449 / 395
2505 / 395
2592 / 395
2617 / 395
2711 / 395
2747 / 395
2749 / 395
3156 / 395
3160 / 395
3161 / 395
3164 / 395
3166 / 395
3172 / 395
3198 / 395
3199 / 395
3202 / 395
3214 / 395
3215 / 395
3225 / 395
3230 / 395
3233 / 395
3242 / 395
3256 / 395
3257 / 395
3262 / 395
3271 / 395
3276 / 395
3286 / 395
3294 / 395
3303 / 395
3321 / 395
3399 / 395
3451 / 395
3467 / 395
3558 / 395
3678 / 395
391 / 395
3969 / 395
3973 

In [5]:
# go through each player and split columns into different player positions

import glob
import json

playerGameFiles = glob.glob("./storage/*")

def openFile(file):
    with open(file, 'r') as readFile:
        return json.load(readFile)

def extractColumns(data):
    if data:
        return list(data['stats'].keys())
    return []

def retrieveColumns(data):
    weeks = list(data.keys())
    columns = list(map(lambda x: extractColumns(data[x]), weeks))
    flatColumns = [j for i in columns for j in i]
    return list(set(flatColumns))

# find position of all players

positionColumnMap = {}

def updatePositionMap(position, newColumns):
    if position in positionColumnMap:
        currentCols = positionColumnMap[position]
        updatedCols = list(set(currentCols + newColumns))
        positionColumnMap[position] = updatedCols
    else:
        positionColumnMap[position] = newColumns
    
for playerGameFileIndex in range(0, len(playerGameFiles)):
    playerGameFile = playerGameFiles[playerGameFileIndex]
    playerId = playerGameFile.split('/')[2].split('.')[0]
    if playerId in players_map:
        position = players_map[playerId][0]
        fileData = openFile(playerGameFile)
        cols = retrieveColumns(fileData)
        updatePositionMap(position, cols)
    else:
        print('Missing ID in Player Map', playerId)


In [29]:
import sqlite3
from sqlite3 import Error


def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file, timeout=10)
    except Error as e:
        print(e)

    return conn

def create_players_table(conn):
    sql = ''' CREATE TABLE player (
                id TEXT PRIMARY KEY,
                firstName TEXT,
                lastName TEXT,
                position TEXT
              );'''
    cur = conn.cursor()
    cur.execute(sql)
    conn.commit()

# SQL INJECTION
# PRIMARY KEY(source_id, author_id, alias_id),
def create_positions_table(conn, tableName, columns):
    pid = ['pid TEXT NOT NULL', 'week INTEGER NOT NULL']
    columns = list(map(lambda x: f'{x} REAL DEFAULT 0 NOT NULL', columns))
    joinedCols = ','.join(pid + columns)
    foreign = 'FOREIGN KEY (pid) REFERENCES player (id)'
    sql = f'CREATE TABLE {tableName} ({joinedCols}, PRIMARY KEY(pid, week) {foreign});'
    cur = conn.cursor()
    cur.execute(sql)
    conn.commit()

# INSERT
# INSERT INTO artists (name)
# VALUES('Bud Powell');
def create_player_week(conn, table, cols, vals):
    joinedCols = ','.join(cols)
    joinedValues = ','.join(list(map(lambda x: str(x), vals)))
    sql = f'INSERT INTO {table} ({joinedCols}) VALUES ({joinedValues});'
    try:
        cur = conn.cursor()
        cur.execute(sql)
        conn.commit()
    except:
        print(vals[0], vals[1], sql)


In [85]:
conn = create_connection("../../api/football.db")
create_players_table(conn)

In [6]:
# NEED TO PROVIDE USERS THE ABILITY TO SEED THE DB
# MIGHT NEED TO MOVE FROM SQLITE3 to POSTGRES

players = pd.read_json('../players.json')
 
def setPos(pos):
    if pos and len(pos) >= 1:
        return pos[0]
    return pos

players['position'] = players.apply(lambda x: setPos(x['positions']), axis=1)
players[['id', 'firstName', 'lastName', 'position']].to_csv('players.csv', index=False)

# positionColumnMap

{'K': ['fgmiss_50p',
  'fg_blkd',
  'fgm_pct',
  'fgm_lng',
  'st_snp',
  'tm_st_snp',
  'gms_active',
  'pos_rank_std',
  'fgmiss_30_39',
  'gp',
  'pos_rank_ppr',
  'fgm_yds',
  'xp_blkd',
  'pts_half_ppr',
  'xpm',
  'fga',
  'fgm_40_49',
  'fgmiss_20_29',
  'tm_off_snp',
  'fgmiss',
  'st_tkl_solo',
  'pos_rank_half_ppr',
  'fgm',
  'fgm_yds_over_30',
  'tm_def_snp',
  'fgmiss_40_49',
  'fgm_50p',
  'fgm_0_19',
  'fgm_20_29',
  'fgm_30_39',
  'xpa',
  'xpmiss',
  'kick_pts',
  'pts_std',
  'pts_ppr'],
 'DEF': ['ff',
  'yds_allow_350_399',
  'fg_blkd',
  'ff_misc',
  'pts_allow_28_34',
  'def_st_ff',
  'pos_rank_std',
  'def_pass_def',
  'penalty',
  'fan_pts_allow_te',
  'def_st_td',
  'def_3_and_out',
  'gp',
  'pos_rank_ppr',
  'pts_allow_35p',
  'int_ret_yd',
  'tkl_solo',
  'pts_half_ppr',
  'def_kr',
  'fan_pts_allow_rb',
  'qb_hit',
  'misc_ret_yd',
  'def_pr_yd',
  'safe',
  'blk_kick_ret_yd',
  'tkl',
  'def_forced_punts',
  'yds_allow_300_349',
  'sack',
  'pts_allow_1_6',

In [8]:
conn = create_connection("../../api/football.db")

for key, value in positionColumnMap.items():
    value.sort()
    create_positions_table(conn, key, value)


In [30]:
# INSERT DATA INTO TABLE
# TODO should i use transitions and rollbacks and commits for each DB transaction...

# go through each player and split columns into different player positions

import glob

playerGameFiles = glob.glob("./storage/*")

def openFile(file):
    with open(file, 'r') as readFile:
        return json.load(readFile)

def extractColumns(data):
    if data:
        return list(data['stats'].keys())
    return []

def retrieveColumns(data):
    weeks = list(data.keys())
    columns = list(map(lambda x: extractColumns(data[x]), weeks))
    flatColumns = [j for i in columns for j in i]
    return list(set(flatColumns))

conn = create_connection("../../api/football.db")

def insertByGame(table, data, pid):
    weeks = list(data.keys())
    for week in weeks:
        if data[week]:
            columns = ['pid','week'] + list(data[week]['stats'].keys())
            values = [f'"{pid}"', int(week)] + list(data[week]['stats'].values())
            create_player_week(conn, table, columns, values)
    
# for playerGameFileIndex in range(0, 1):
for playerGameFileIndex in range(1, len(playerGameFiles)):
    playerGameFile = playerGameFiles[playerGameFileIndex]
    playerId = playerGameFile.split('/')[2].split('.')[0]
    if playerId in players_map:
        position = players_map[playerId][0]
        fileData = openFile(playerGameFile)
        insertByGame(position, fileData, playerId)
    else:
        print('Missing ID in Player Map', playerId)